In [1]:
from __future__ import division
from __future__ import print_function

seed_value= 0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.set_random_seed(seed_value)
from keras import backend as K
import os
import keras
import pickle
import os.path
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelBinarizer
from keras import initializers
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import time
from tqdm import tqdm

import six.moves.cPickle as pickle
import gzip
import os
import sys
import timeit

Using TensorFlow backend.


In [2]:
dataset='mnist.pkl.gz'
batch_size = 128
num_classes = 10
epochs = 100
lr = 0.1
lr2 = 0.02
lr3 = 0.7
eps = 0.1
#K.clear_session()

In [3]:
print('... loading data')

# Load the dataset
with gzip.open(dataset, 'rb') as f:
    try:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    except:
        train_set, valid_set, test_set = pickle.load(f)

... loading data


In [4]:
(x_train, y_train) = train_set
(x_test, y_test) = test_set
(x_val, y_val) = valid_set
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 28, 28, 1)
50000 train samples
10000 validation samples
10000 test samples


In [5]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((50000, 28, 28, 1),
 (50000, 10),
 (10000, 28, 28, 1),
 (10000, 10),
 (10000, 28, 28, 1),
 (10000, 10))

In [6]:
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_val = np.pad(x_val, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((50000, 32, 32, 1),
 (50000, 10),
 (10000, 32, 32, 1),
 (10000, 10),
 (10000, 32, 32, 1),
 (10000, 10))

In [7]:
input_shape = (32, 32, 1)

In [8]:
def build_model(setseed):
    """
    Builds test Keras model for LeNet MNIST
    :param loss (str): Type of loss - must be one of Keras accepted keras losses
    :return: Keras dense model of predefined structure
    """
    input = Input(shape=input_shape)
    conv1 = Conv2D(6, (3,3), activation='relu', kernel_initializer=initializers.lecun_uniform(seed = setseed))(input)
    avg1 = AveragePooling2D()(conv1)
    conv2 = Conv2D(16, (3,3), activation='relu', kernel_initializer=initializers.lecun_uniform(seed = setseed))(avg1)
    avg2 = AveragePooling2D()(conv2)
    flat= Flatten()(avg2)
    dens1=Dense(units=120, activation='relu')(flat)
    dens2=Dense(units=84, activation='relu')(dens1)
    probs=Dense(num_classes, activation='softmax')(dens2)
    
    model = Model(input=input, output=probs)
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics = ['accuracy'])

    return model



In [9]:
all_model = [None,None,None]
losses = [None,None,None]

prediction=[]

all_score =[0,0,0]
gr=[]
wr=[]
xwr=[]

for i in range(3):
    np.random.seed(25+i)
    model = build_model(i+2)
    all_model[i]=model
    
for i in range(3):    
    weights = all_model[i].trainable_weights # weight tensors
    weights = [weight for weight in weights] # filter down weights tensors to only ones which are trainable
    gradients = all_model[i].optimizer.get_gradients(all_model[i].total_loss, weights) # gradient tensors
    gr.append(gradients)
    wr.append(weights)
    xweights = all_model[i].non_trainable_weights # weight tensors
    xweights = [weight for weight in xweights] # filter down weights tensors to only ones which are trainable
    xwr.append(xweights)

    losses[i]=all_model[i].total_loss
    prediction.append(all_model[i].output)

C:\Users\ikarabayir\AppData\Local\Continuum\anaconda2\envs\tf\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 6)         60        
_________________________________________________________________
average_pooling2d_5 (Average (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
average_pooling2d_6 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 120)               69240     
__________

In [11]:
input_tensors = [all_model[0].inputs[0], # input data
                 all_model[0].sample_weights[0], # how much to weight each sample by
                 all_model[0].targets[0], # labels
                 K.learning_phase(), # train or test mode
                 all_model[1].inputs[0], # input data
                 all_model[1].sample_weights[0], # how much to weight each sample by
                 all_model[1].targets[0], # labels
                 all_model[2].inputs[0], # input data
                 all_model[2].sample_weights[0], # how much to weight each sample by
                 all_model[2].targets[0], # labels
                ]


minlos = K.argmin(losses)

grr=[]
for x in gr:
    for y in x:
        grr.append(y)

upd_test= K.function(inputs=input_tensors, outputs=[ losses[0], losses[1], losses[2], minlos, prediction[0], prediction[1], prediction[2] ])


grad_best=[]
grad_non0 = []
grad_non1 = []


weig_best=[]
weig_non0 = []
weig_non1 = []

xweig_best=[]
xweig_non0 = []
xweig_non1 = []




for i in range(len(gr[0])):
    gr_ck=tf.concat([gr[0][i],gr[1][i], gr[2][i]],0)
    newshape = (3, ) + (tuple(wr[0][i].shape))

    
    gr_ck2=tf.reshape(gr_ck, newshape)
    
    bb = gr_ck2[minlos]
    grad_best.append(bb)
    
    nbb0 = gr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    nbb1 = gr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    nbc = tf.concat([nbb0,nbb1], 0)    
    nbc = tf.reshape(nbc, (-1,))
    newshape2 = (2, ) + (tuple(wr[0][i].shape))
    
    nbc2 = tf.reshape(nbc, newshape2) 
    nb0 = nbc2[0]
    nb1 = nbc2[1]
    grad_non0.append(nb0)
    grad_non1.append(nb1)
    

    wr_ck=tf.concat([wr[0][i],wr[1][i], wr[2][i]],0)
    
    newshape = (3, ) + (tuple(wr[0][i].shape))
    wr_ck2=tf.reshape(wr_ck, newshape) 
    bb2 = wr_ck2[minlos]
    weig_best.append(bb2)
    
    #wb = wr_ck[minlos]
    wnbb0 = wr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    wnbb1 = wr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    wnbc = tf.concat([wnbb0,wnbb1],0)    
    wnbc = tf.reshape(wnbc, (-1,))
    newshape2 = (2, ) + (tuple(wr[0][i].shape))
    
    wnbc2 =tf.reshape(wnbc, newshape2)
    wnb0 = wnbc2[0]
    wnb1 = wnbc2[1]
    weig_non0.append(wnb0)
    weig_non1.append(wnb1)
    
    if i<len(xwr[0]):
        print (i)
        xwr_ck=tf.concat([xwr[0][i],xwr[1][i], xwr[2][i]], 0)

        newshape = (3, ) + (tuple(xwr[0][i].shape))
        
        xwr_ck2=tf.reshape(xwr_ck, newshape)  
        xbb2 = xwr_ck2[minlos]
        xweig_best.append(xbb2)

        #wb = wr_ck[minlos]
        xwnbb0 = xwr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
        xwnbb1 = xwr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
        xwnbc = tf.concat([xwnbb0,xwnbb1], 0)    
        
        xwnbc = tf.reshape(xwnbc, (-1,))
        newshape2 = (2, ) + (tuple(xwr[0][i].shape))
         
        xwnbc2 = tf.reshape(xwnbc, newshape2) 
        xwnb0 = xwnbc2[0]
        xwnb1 = xwnbc2[1]
        xweig_non0.append(xwnb0)
        xweig_non1.append(xwnb1)
    else:
        pass

    
los=tf.stack([losses[0], losses[1], losses[2]])

newshape = (3, )
los2=tf.reshape(los, newshape) 
losbest = los2[minlos]

#wb = wr_ck[minlos]
los_0 = los2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
los_1 = los2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
loswnbc = tf.concat([los_0,los_1],0)    
loswnbc = tf.reshape(loswnbc,(-1,))
newshape2 = (2, )

loswnbc2 = tf.reshape(loswnbc, newshape2)
losss0 = loswnbc2[0]
losss1 = loswnbc2[1]




mn0 = [tf.keras.backend.l2_normalize((best-nonbest)*(losbest-losss0)/ tf.reduce_sum(tf.pow((best-nonbest),2)+eps))  for best, nonbest in zip(weig_best, weig_non0)]

mn1 = [tf.keras.backend.l2_normalize((best-nonbest)*(losbest-losss1)/ tf.reduce_sum(tf.pow((best-nonbest),2)+eps))  for best, nonbest in zip(weig_best, weig_non1)]



nCom0 = [non- lr3* grad - lr2* mn for mn, grad, non in zip(mn0,grad_non0, weig_non0 )]

nCom1 = [non- lr3* grad - lr2 * mn for mn, grad, non in zip(mn1,grad_non1, weig_non1 )]

xbest = [ -lr * nc + non for nc, non in zip(grad_best, weig_best)]


upd2 = [
    tf.assign(param_i, v)
    for param_i, v in zip(wr[2], xbest)
]

upd2.extend(
        [tf.assign(param_i, v)
        for param_i, v in zip(xwr[2], xweig_best)]
    )

upd2.extend(
        [tf.assign(param_i, v)
        for param_i, v in zip(wr[1], nCom0)]
    )
upd2.extend(
        [tf.assign(param_i, v)
        for param_i, v in zip(xwr[1], xweig_non0)]
    )
upd2.extend(
        [tf.assign(param_i, v)
        for param_i, v in zip(wr[0], nCom1)]
    )
upd2.extend(
        [tf.assign(param_i, v)
        for param_i, v in zip(xwr[0], xweig_non1)]
    )


upd_bb2= K.function(inputs=input_tensors, outputs=[ losses[0], losses[1], losses[2], minlos, prediction[0], prediction[1], prediction[2] ], updates=upd2)





lossepoch=[]
lossepoch_test=[]
lossx=[]
acctra=[]
loss_test=[]
acc_test=[]
skip=[]


loss_val=[]
acc_val=[]
lossepoch_val=[]

   
for f in tqdm(range(epochs)):
    program_starts = time.time()
    tr1=[]
    tr2=[]
    res1=[]
    res2=[]
    res3=[]
    res4=[]
    print('Epoch', f)
    print ('train')
    
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:

    batches = 0
    for x_batch, y_batch in ImageDataGenerator().flow(x_train, y_train, batch_size=batch_size):
        K.set_learning_phase(1)
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in Train mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_bb2(inputs)
        yhat=ll[6]
        #print (accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        #print (ll[:4])
        lossepoch.append(ll[2])
        tr1.append(ll[2])
        tr2.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        skip.append(ll[3])
        batches += 1
        if batches > len(x_train) / batch_size:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    m=(len(x_train) / batch_size)-int((len(x_train) / batch_size))
    tr1[-1]*=m
    tr2[-1]*=m
    lossx.append(np.mean(tr1))
    acctra.append(np.mean(tr2))
    print ('train loss score is :'+str(np.mean(tr1)))
    print ('train acc score is :'+str(np.mean(tr2)))
    now = time.time()
    print("It has been {0} seconds since the loop started".format(now - program_starts))
    print ('test')
    batchesx = 0
    
    
    
    
    print ('validation')
    batchesx = 0
    for x_batch, y_batch in ImageDataGenerator().flow(x_val, y_val, batch_size=batch_size):
        K.set_learning_phase(0)
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in VAl mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_test(inputs)
        yhat=ll[6]
        #print (accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        #print (ll[:3])
        lossepoch_val.append(ll[2])
        res3.append(ll[2])
        res4.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        batchesx += 1
        if batchesx >= len(x_val) / batch_size:
            break
    m=(len(x_val) / batch_size)-int((len(x_val) / batch_size))
    res3[-1]*=m
    res4[-1]*=m
    loss_val.append(np.mean(res3))
    acc_val.append(np.mean(res4))
    print ('val loss score is :'+str(np.mean(res3)))
    print ('val acc score is :'+str(np.mean(res4)))
    print ('test')
    batchesx = 0
    for x_batch, y_batch in ImageDataGenerator().flow(x_test, y_test, batch_size=batch_size):
        K.set_learning_phase(0)
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in TEST mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_test(inputs)
        yhat=ll[6]
        #print (accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        #print (ll[:3])
        lossepoch_test.append(ll[2])
        res1.append(ll[2])
        res2.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        batchesx += 1
        if batchesx >= len(x_test) / batch_size:
            break
    m=(len(x_test) / batch_size)-int((len(x_test) / batch_size))
    res1[-1]*=m
    res2[-1]*=m
    loss_test.append(np.mean(res1))
    acc_test.append(np.mean(res2))
    print ('test loss score is :'+str(np.mean(res1)))
    print ('test acc score is :'+str(np.mean(res2)))



  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Epoch 0
train
Using real-time data augmentation.
train loss score is :0.3922014174782826
train acc score is :0.8760390025575447
It has been 9.605804681777954 seconds since the loop started
test
validation
val loss score is :0.15452594646618148
val acc score is :0.9406645569620253
test
test loss score is :0.15095577745026426
test acc score is :0.9408623417721519


  1%|▊                                                                                 | 1/100 [00:11<18:20, 11.12s/it]

Epoch 1
train
Using real-time data augmentation.
train loss score is :0.14550746086022587
train acc score is :0.9544637148337596
It has been 6.017298460006714 seconds since the loop started
test
validation
val loss score is :0.15083223409196245
val acc score is :0.9442246835443038
test
test loss score is :0.15176973952852849
test acc score is :0.9449169303797469


  2%|█▋                                                                                | 2/100 [00:18<16:20, 10.00s/it]

...

 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [12:36<00:16,  8.09s/it]

Epoch 98
train
Using real-time data augmentation.
train loss score is :9.753931872792015e-06
train acc score is :0.9990409207161125
It has been 6.534330129623413 seconds since the loop started
test
validation
val loss score is :0.06589610477963187
val acc score is :0.9798259493670886
test
test loss score is :0.05598962460248052
test acc score is :0.9807159810126582


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [12:44<00:08,  8.05s/it]

Epoch 99
train
Using real-time data augmentation.
train loss score is :9.696708010523825e-06
train acc score is :0.9990409207161125
It has been 6.554347515106201 seconds since the loop started
test
validation
val loss score is :0.06601101487062144
val acc score is :0.9798259493670886
test
test loss score is :0.05606305247760151
test acc score is :0.9807159810126582


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [12:52<00:00,  8.03s/it]


In [12]:
#\K.clear_session()

In [14]:
print (np.max(acc_test), np.argmax(acc_test))
print (np.max(acc_val), np.argmax(acc_val))
print (np.min(loss_test), np.argmin(loss_test))
print (np.min(loss_val), np.argmin(loss_val))


0.9812104430379747 30
0.9806170886075949 23
0.030497980957609142 9
0.03898478883638745 9


In [15]:
np.savetxt("CR2_EVGO3_Mnist_lossepoch.csv", lossepoch, delimiter=",", fmt='%s')
np.savetxt("CR2_EVGO3_Mnist_lossepoch_test.csv", lossepoch_test, delimiter=",", fmt='%s')
np.savetxt("CR2_EVGO3_Mnist_loss_tra.csv", lossx, delimiter=",", fmt='%s')
np.savetxt("CR2_EVGO3_Mnist_skip.csv", skip, delimiter=",", fmt='%s')
np.savetxt("CR2_EVGO3_Mnist_loss_test.csv", loss_test, delimiter=",", fmt='%s')
np.savetxt("CR2_EVGO3_Mnist_acc_test.csv", acc_test, delimiter=",", fmt='%s')

In [16]:
for i in range(len(skip)):
    if skip[i]==0:
        print (i)

0
9
229
405
418
439
440
453
454
461
462
475
516
521
540
563
592
613
634
749
780
785
786
789
797
798
810
815
822
823
828
829
832
833
836
837
838
841
846
851
858
865
877
878
879
884
887
890
902
903
904
905
909
914
917
922
929
930
940
941
945
972
975
976
977
980
983
984
985
988
994
1003
1004
1005
1008
1010
1012
1021
1022
1027
1030
1037
1052
1053
1054
1062
1063
1065
1068
1070
1075
1078
1085
1095
1096
1101
1106
1107
1109
1116
1117
1119
1120
1121
1124
1126
1131
1132
1138
1139
1142
1143
1145
1146
1150
1151
1159
1161
1162
1163
1164
1167
1172
1176
1177
1180
1182
1189
1190
1193
1197
1198
1202
1203
1204
1207
1208
1210
1223
1224
1227
1231
1233
1239
1241
1242
1245
1250
1252
1254
1255
1256
1260
1262
1264
1266
1267
1274
1276
1282
1283
1293
1297
1302
1303
1304
1306
1309
1311
1314
1318
1320
1321
1323
1329
1336
1342
1348
1349
1365
1370
1371
1375
1384
1387
1393
1400
1404
1407
1410
1411
1416
1420
1422
1430
1431
1436
1437
1438
1445
1446
1448
1454
1455
1456
1457
1458
1464
1465
1471
1474
1475
1476
1477
1480


KeyboardInterrupt: 

In [17]:
for i in range(len(skip)):
    if skip[i]==1:
        print (i)

1
446
460
464
474
527
556
565
569
573
583
587
600
608
615
782
788
791
793
814
821
827
831
845
850
853
862
863
866
870
886
913
916
919
921
928
944
949
951
967
971
974
987
989
990
1000
1002
1007
1011
1013
1016
1017
1018
1019
1023
1026
1029
1033
1036
1051
1056
1058
1059
1060
1064
1066
1067
1073
1074
1086
1087
1089
1090
1091
1092
1097
1100
1103
1105
1115
1118
1129
1134
1158
1165
1166
1173
1174
1178
1179
1181
1195
1196
1199
1200
1205
1206
1209
1236
1249
1251
1253
1258
1259
1263
1265
1268
1269
1275
1277
1278
1295
1305
1307
1310
1312
1315
1317
1324
1326
1327
1328
1331
1332
1334
1335
1338
1339
1343
1344
1350
1352
1363
1367
1368
1373
1374
1381
1386
1389
1390
1391
1392
1403
1406
1408
1412
1417
1423
1426
1427
1429
1433
1435
1449
1451
1452
1453
1460
1470
1473
1478
1486
1488
1490
1502
1519
1520
1526
1528
1535
1543
1547
1551
1555
1563
1568
1570
1572
1576
1579
1580
1583
1584
1587
1588
1591
1593
1605
1608
1620
1625
1632
1633
1653
1655
1661
1663
1665
1671
1673
1682
1685
1697
1704
1706
1708
1710
1716
17

KeyboardInterrupt: 